In [ ]:
import numpy as np
import pandas as pd
import json
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score
import shap
import warnings

method = 'TCP_LT_AOF'
data = pd.read_csv(f'../input_data/{method}/data.csv')

with open(f'../results/{method}/final_hyperparameters.json', 'r') as file:
    best_params = json.load(file)

parameters = ['normal_convergence_rate', 
              'subducting_ocean_floor_age',
              'obliquity_of_subduction',
              'migration_rate_x_distance']

X = data[parameters]
y = data['cu_mt']
y_cat = np.where(y > 2, 1, 0)

np.random.seed(42)
CV_iterations = 1000
random_states = np.random.randint(9999, size=CV_iterations)

# Initialize dictionaries with proper structure
shap_values_per_cv = dict()
for sample in X.index:
    shap_values_per_cv[sample] = {}
    for CV_iteration in range(CV_iterations):
        shap_values_per_cv[sample][CV_iteration] = {}

# Lists to store the performance metrics (calculated from aggregated predictions)
roc_auc_scores, precision_scores, recall_scores, f1_score_scores = [], [], [], []

# Dictionary to store confusion matrices for each CV iteration
confusion_matrices_per_cv = {}

# Repeated cross-validations
for i, CV_iteration in enumerate(range(CV_iterations)):
    print('\n------------ CV Repeat number:', CV_iteration)
    
    CV = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_states[i])
    
    pipeline = Pipeline(steps=[
        ('model', XGBClassifier(seed=42))
    ])
    
    # Store training and test indices
    ix_training, ix_test = [], []
    for fold in CV.split(X, y_cat):
        ix_training.append(fold[0]), ix_test.append(fold[1])
    
    # Initialize lists to collect all predictions and true labels for this CV iteration
    all_y_true = []
    all_y_pred = []
    all_y_pred_proba = []  # For AUC calculation
    
    # Process each fold
    for fold_idx, (train_outer_ix, test_outer_ix) in enumerate(zip(ix_training, ix_test)):
        X_train, X_test = X.iloc[train_outer_ix, :], X.iloc[test_outer_ix, :]
        y_train, y_test = y_cat[train_outer_ix], y_cat[test_outer_ix]
        
        pipeline.set_params(**best_params)
        fit = pipeline.fit(X_train, y_train)
        
        # Get predictions for confusion matrix and metrics
        y_pred = fit.predict(X_test)
        y_pred_proba = fit.predict_proba(X_test)[:, 1]  # Probability of positive class
        
        # Collect predictions and true labels
        all_y_true.extend(y_test)
        all_y_pred.extend(y_pred)
        all_y_pred_proba.extend(y_pred_proba)
        
        model = fit.named_steps['model']
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)
        
        # Store SHAP values and predictions for each test sample
        for j, test_index in enumerate(test_outer_ix):
            shap_values_per_cv[test_index][CV_iteration] = shap_values[j]
    
    # Create confusion matrix for this CV iteration (aggregated across all folds)
    cm = confusion_matrix(all_y_true, all_y_pred)
    confusion_matrices_per_cv[CV_iteration] = cm
    
    # Calculate all metrics from aggregated predictions
    # AUC from probability predictions
    roc_auc = roc_auc_score(all_y_true, all_y_pred_proba)
    roc_auc_scores.append(roc_auc)
    
    # Other metrics from confusion matrix
    tn, fp, fn, tp = cm.ravel()
    
    # Precision for positive class (High-Cu deposit detection)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    precision_scores.append(precision)
    
    # Recall for positive class (High-Cu deposit detection)  
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    recall_scores.append(recall)
    
    # F1-score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f1_score_scores.append(f1)

# Save performance metrics
performance_metrics_df = pd.DataFrame({
    'roc_auc': roc_auc_scores,
    'precision': precision_scores,
    'recall': recall_scores,
    'f1_score': f1_score_scores
})

performance_metrics_df.to_csv('performance_metrics.csv', index=False)

# Save SHAP values for each sample
for n in range(len(data)):
    shaps_per_obs = pd.DataFrame.from_dict(shap_values_per_cv[n])
    shaps_per_obs.to_csv(f'sample_{n}.csv', index=False)